In [ ]:
!pip install pandas_ta

In [8]:
import pandas as pd
import requests

url = 'https://en.wikipedia.org/wiki/List_of_S&P_500_companies'

# Add headers to look like a normal web browser
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'}

# Fetch the page manually
response = requests.get(url, headers=headers)
response.raise_for_status()  # Raises an error if the request failed

# Now read the HTML from the page content
sp500 = pd.read_html(response.text)[0]

print(sp500.head())


  Symbol             Security             GICS Sector  \
0    MMM                   3M             Industrials   
1    AOS          A. O. Smith             Industrials   
2    ABT  Abbott Laboratories             Health Care   
3   ABBV               AbbVie             Health Care   
4    ACN            Accenture  Information Technology   

                GICS Sub-Industry    Headquarters Location  Date added  \
0        Industrial Conglomerates    Saint Paul, Minnesota  1957-03-04   
1               Building Products     Milwaukee, Wisconsin  2017-07-26   
2           Health Care Equipment  North Chicago, Illinois  1957-03-04   
3                   Biotechnology  North Chicago, Illinois  2012-12-31   
4  IT Consulting & Other Services          Dublin, Ireland  2011-07-06   

       CIK      Founded  
0    66740         1902  
1    91142         1916  
2     1800         1888  
3  1551152  2013 (1888)  
4  1467373         1989  


In [22]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

# sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2025-10-10'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()

df

[*********************100%***********************]  503 of 503 completed


Price                   close        high         low        open      volume
date       ticker                                                            
2017-10-12 A        63.584099   63.640582   62.642670   62.746228   1455000.0
           AAPL     36.438347   36.758349   36.375279   36.520101  64500400.0
           ABBV     65.404686   65.812481   65.039820   65.655086   4818200.0
           ABT      47.746803   47.764258   47.223169   47.336625   4405000.0
           ACGL     31.237015   31.363803   31.157774   31.335277   1103100.0
...                       ...         ...         ...         ...         ...
2025-10-09 XYZ      80.849998   81.500000   80.010002   81.089996   4193600.0
           YUM     142.690002  146.300003  142.679993  145.360001   1371000.0
           ZBH      97.370003   98.879997   97.050003   98.680000    810000.0
           ZBRA    298.790009  308.269989  294.790009  307.190002    645500.0
           ZTS     143.389999  145.220001  143.250000  143.820007   3511000.0

[991645 rows x 5 columns]

In [23]:
df_ = df
df_.head()

Price                  close       high        low       open      volume
date       ticker                                                        
2017-10-12 A       63.584099  63.640582  62.642670  62.746228   1455000.0
           AAPL    36.438347  36.758349  36.375279  36.520101  64500400.0
           ABBV    65.404686  65.812481  65.039820  65.655086   4818200.0
           ABT     47.746803  47.764258  47.223169  47.336625   4405000.0
           ACGL    31.237015  31.363803  31.157774  31.335277   1103100.0

2. Calculate features and technical indicators for each stock.
Garman-Klass Volatility
RSI
Bollinger Bands
ATR
MACD
Dollar Volume

In [24]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

df['bb_mid'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

df['bb_high'] = df.groupby(level=1)['close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['close'].apply(compute_macd)

df['dollar_volume'] = (df['close']*df['volume'])/1e6

df

Price                   close        high         low        open      volume  \
date       ticker                                                               
2017-10-12 A        63.584099   63.640582   62.642670   62.746228   1455000.0   
           AAPL     36.438347   36.758349   36.375279   36.520101  64500400.0   
           ABBV     65.404686   65.812481   65.039820   65.655086   4818200.0   
           ABT      47.746803   47.764258   47.223169   47.336625   4405000.0   
           ACGL     31.237015   31.363803   31.157774   31.335277   1103100.0   
...                       ...         ...         ...         ...         ...   
2025-10-09 XYZ      80.849998   81.500000   80.010002   81.089996   4193600.0   
           YUM     142.690002  146.300003  142.679993  145.360001   1371000.0   
           ZBH      97.370003   98.879997   97.050003   98.680000    810000.0   
           ZBRA    298.790009  308.269989  294.790009  307.190002    645500.0   
           ZTS     143.389999  145.220001  143.250000  143.820007   3511000.0   

Price              garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2017-10-12 A               0.000057        NaN       NaN       NaN       NaN   
           AAPL            0.000053        NaN       NaN       NaN       NaN   
           ABBV            0.000064        NaN       NaN       NaN       NaN   
           ABT             0.000036        NaN       NaN       NaN       NaN   
           ACGL            0.000018        NaN       NaN       NaN       NaN   
...                             ...        ...       ...       ...       ...   
2025-10-09 XYZ             0.000167  60.379617  4.283941  4.346427  4.408912   
           YUM             0.000181  40.952681  4.972897  5.011208  5.049518   
           ZBH             0.000105  43.807362  4.579586  4.608027  4.636468   
           ZBRA            0.000703  44.795905  5.666716  5.729912  5.793109   
           ZTS             0.000090  40.839835  4.954437  4.983381  5.012324   

Price                   atr      macd  dollar_volume  
date       ticker                                     
2017-10-12 A            NaN       NaN      92.514864  
           AAPL         NaN       NaN    2350.287948  
           ABBV         NaN       NaN     315.132858  
           ABT          NaN       NaN     210.324668  
           ACGL         NaN       NaN      34.457551  
...                     ...       ...            ...  
2025-10-09 XYZ    -0.695561  0.182344     339.052554  
           YUM     0.841665 -0.206738     195.627993  
           ZBH    -0.754498 -0.355211      78.869702  
           ZBRA    0.028098 -0.574604     192.868951  
           ZTS    -0.307528 -0.782493     503.442288  

[991645 rows x 13 columns]

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 991645 entries, (Timestamp('2017-10-12 00:00:00'), 'A') to (Timestamp('2025-10-09 00:00:00'), 'ZTS')
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   close             991645 non-null  float64
 1   high              991645 non-null  float64
 2   low               991645 non-null  float64
 3   open              991645 non-null  float64
 4   volume            991645 non-null  float64
 5   garman_klass_vol  991645 non-null  float64
 6   rsi               991104 non-null  float64
 7   bb_low            982088 non-null  float64
 8   bb_mid            982088 non-null  float64
 9   bb_high           982088 non-null  float64
 10  atr               985106 non-null  float64
 11  macd              979070 non-null  float64
 12  dollar_volume     991645 non-null  float64
dtypes: float64(13)
memory usage: 102.2+ MB


Aggregate to monthly level and filter top 150 most liquid stocks for each month.
To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [26]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

data.shape

(47383, 8)

Calculate 5-year rolling average of dollar volume for each stocks before filtering.

In [27]:
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2018-10-31 AAPL            0.000122  50.232953  3.931802  3.971160  4.010518   
           ABBV            0.000562  30.334060  4.062166  4.186937  4.311708   
           ABT             0.000197  51.539001  4.088700  4.131786  4.174872   
           ACN             0.000204  44.756049  4.910355  4.976741  5.043127   
           ADBE            0.000288  46.518481  5.451019  5.520491  5.589962   
...                             ...        ...       ...       ...       ...   
2025-10-31 WDAY            0.000144  53.107662  5.402193  5.467197  5.532202   
           WFC             0.000138  45.202000  4.379241  4.424490  4.469739   
           WMT             0.000389  51.177328  4.630167  4.643197  4.656227   
           XOM             0.000226  51.402562  4.716948  4.742014  4.767080   
           XYZ             0.000167  60.379617  4.283941  4.346427  4.408912   

                        atr      macd  
date       ticker                      
2018-10-31 AAPL   -0.865174 -0.398819  
           ABBV   -0.430494 -1.461402  
           ABT    -0.654055 -0.493491  
           ACN    -0.837395 -0.678882  
           ADBE   -0.008434 -0.539863  
...                     ...       ...  
2025-10-31 WDAY    0.325781  0.291039  
           WFC     1.225222 -0.336949  
           WMT     1.845518  0.243291  
           XOM     0.616412  0.264528  
           XYZ    -0.695561  0.182344  

[12665 rows x 7 columns]

Calculate Monthly Returns for different time horizons as features.
To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.

In [31]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df

# Calculate returns on the original daily data
returns_df = df.groupby(level=1, group_keys=False).apply(calculate_returns)

# Aggregate returns to monthly level and merge with the existing data
for lag in [1, 2, 3, 6, 9, 12]:
    data[f'return_{lag}m'] = returns_df[f'return_{lag}m'].unstack('ticker').resample('M').last().stack('ticker')

data = data.dropna()

data

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2018-10-31 AAPL            0.000122  50.232953  3.931802  3.971160  4.010518   
           ABBV            0.000562  30.334060  4.062166  4.186937  4.311708   
           ABT             0.000197  51.539001  4.088700  4.131786  4.174872   
           ACN             0.000204  44.756049  4.910355  4.976741  5.043127   
           ADBE            0.000288  46.518481  5.451019  5.520491  5.589962   
...                             ...        ...       ...       ...       ...   
2025-10-31 WDAY            0.000144  53.107662  5.402193  5.467197  5.532202   
           WFC             0.000138  45.202000  4.379241  4.424490  4.469739   
           WMT             0.000389  51.177328  4.630167  4.643197  4.656227   
           XOM             0.000226  51.402562  4.716948  4.742014  4.767080   
           XYZ             0.000167  60.379617  4.283941  4.346427  4.408912   

                        atr      macd  return_1m  return_2m  return_3m  \
date       ticker                                                        
2018-10-31 AAPL   -0.865174 -0.398819   0.026067   0.015476   0.003930   
           ABBV   -0.430494 -1.461402  -0.045839  -0.021809  -0.012280   
           ABT    -0.654055 -0.493491   0.013823   0.012036   0.009610   
           ACN    -0.837395 -0.678882   0.019403   0.015849   0.006795   
           ADBE   -0.008434 -0.539863   0.030829   0.012456  -0.000054   
...                     ...       ...        ...        ...        ...   
2025-10-31 WDAY    0.325781  0.291039  -0.000126   0.010771  -0.000196   
           WFC     1.225222 -0.336949   0.002007  -0.007733  -0.003316   
           WMT     1.845518  0.243291  -0.010982  -0.007145  -0.003028   
           XOM     0.616412  0.264528  -0.009735  -0.005925  -0.003780   
           XYZ    -0.695561  0.182344  -0.003206   0.011513   0.012987   

                   return_6m  return_9m  return_12m  
date       ticker                                    
2018-10-31 AAPL    -0.002917   0.001452    0.000573  
           ABBV    -0.010540  -0.015863   -0.011259  
           ABT      0.000897   0.001298    0.000291  
           ACN     -0.000496   0.001006    0.000600  
           ADBE    -0.004012  -0.002054    0.002674  
...                      ...        ...         ...  
2025-10-31 WDAY     0.004268  -0.003865   -0.000731  
           WFC     -0.002030  -0.006878   -0.004536  
           WMT     -0.000311  -0.001506   -0.000612  
           XOM      0.001365  -0.004154   -0.000764  
           XYZ      0.016242   0.010387    0.004576  

[12665 rows x 13 columns]

In [33]:
data.tail()

garman_klass_vol        rsi    bb_low    bb_mid   bb_high  \
date       ticker                                                              
2025-10-31 WDAY            0.000144  53.107662  5.402193  5.467197  5.532202   
           WFC             0.000138  45.202000  4.379241  4.424490  4.469739   
           WMT             0.000389  51.177328  4.630167  4.643197  4.656227   
           XOM             0.000226  51.402562  4.716948  4.742014  4.767080   
           XYZ             0.000167  60.379617  4.283941  4.346427  4.408912   

                        atr      macd  return_1m  return_2m  return_3m  \
date       ticker                                                        
2025-10-31 WDAY    0.325781  0.291039  -0.000126   0.010771  -0.000196   
           WFC     1.225222 -0.336949   0.002007  -0.007733  -0.003316   
           WMT     1.845518  0.243291  -0.010982  -0.007145  -0.003028   
           XOM     0.616412  0.264528  -0.009735  -0.005925  -0.003780   
           XYZ    -0.695561  0.182344  -0.003206   0.011513   0.012987   

                   return_6m  return_9m  return_12m  
date       ticker                                    
2025-10-31 WDAY     0.004268  -0.003865   -0.000731  
           WFC     -0.002030  -0.006878   -0.004536  
           WMT     -0.000311  -0.001506   -0.000612  
           XOM      0.001365  -0.004154   -0.000764  
           XYZ      0.016242   0.010387    0.004576